<a href="https://colab.research.google.com/github/alliarnold/llm-fall-2023/blob/main/MultiClass_Bert_Magic20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
#tqdm is a progress bar
from tqdm import tqdm
from transformers import BertTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

magic_20 = pd.read_csv('/content/drive/My Drive/GC-CUNY/2023_Fall/20answerdata.csv')
magic_20.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,alli_q,answer_20
0,Am I a bad communicator?,"Reply hazy, try again"
1,Am I a bad dancer?,Ask again later
2,Am I a bad listener?,Better not tell you now
3,Am I a good person?,Cannot predict now
4,Am I allergic to peanuts?,"Reply hazy, try again"


In [ ]:
def c2n(answer):
  if answer=="It is certain":
    return 0
  elif answer=="It is decidedly so":
    return 1
  elif answer=="Without a doubt":
    return 2
  elif answer=="Yes definitely":
    return 3
  elif answer=="You may rely on it": #
    return 4
  elif answer=="As I see it, yes":
    return 5
  elif answer=="Most likely":
    return 6
  elif answer=="Outlook good":
    return 7
  elif answer=="Yes": #
    return 8
  elif answer=="Signs point to yes":
    return 9
  elif answer=="Reply hazy, try again":
    return 10
  elif answer=="Ask again later":
    return 11
  elif answer=="Better not tell you now":
    return 12
  elif answer=="Cannot predict now":
    return 13
  elif answer=="Concentrate and ask again":
    return 14
  elif answer=="Don't count on it":
    return 15
  elif answer=="My reply is no":
    return 16
  elif answer=="My sources say no":
    return 17
  elif answer=="Outlook not so good":
    return 18
  else:
    return 19

magic_20['answer_20'] = magic_20['answer_20'].apply(c2n)
magic_20.head()


* It is certain// 0
* It is decidedly so// 1
* Without a doubt// 2
* Yes definitely// 3
* You may rely on it//4
* As I see it, yes//5
* Most likely//6
* Outlook good//7
* Yes//8
* Signs point to yes//9
* Reply hazy, try again"/10
* Ask again later//11
* Better not tell you now//12
* Cannot predict now//13
* Concentrate and ask again//14
* Don't count on it//15
* My reply is no//16
* My sources say no//17
* Outlook not so good//18
* Very doubtful


In [ ]:
magic_20['answer_20'].value_counts()

In [ ]:
magic_20.info()

In [ ]:
magic_20 = magic_20.sample(frac = 1) # shuffling order
magic_20[:100]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
token = tokenizer.encode_plus(
    magic_20['alli_q'].iloc[0],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors="tf"
)

In [ ]:
X_input_ids = np.zeros((len(magic_20), 256))
X_attn_masks = np.zeros((len(magic_20), 256))

In [ ]:
X_input_ids.shape # checks

(1000, 256)

In [ ]:
def generate_training_data(magic_20, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(magic_20['alli_q'])):
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    ids[i,:] = tokenized_text.input_ids
    masks[i,:] = tokenized_text.attention_mask
  return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(magic_20, X_input_ids, X_attn_masks, tokenizer)

1000it [00:00, 1271.37it/s]


In [ ]:
labels = np.zeros((len(magic_20), 20))

In [ ]:
labels.shape

(1000, 20)

In [ ]:
labels[np.arange(len(magic_20)), magic_20['answer_20'].values] = 1

In [ ]:
labels # checking

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [ ]:
dataset.take(1) # more checks

In [ ]:
def SentimentDatasetMapFunctions(input_ids, attn_masks, labels):
  return {
      'input_ids': input_ids,
      'attention_mask': attn_masks
  }, labels

In [ ]:
dataset = dataset.map(SentimentDatasetMapFunctions)

In [ ]:
dataset = dataset.shuffle(10000).batch(8, drop_remainder=True)

In [ ]:
# using 80% for training, 20% for validation
train_dataset = dataset.take(int((len(magic_20)//8)*.8))
val_dataset= dataset.skip(int((len(magic_20)//8)*.2))

In [ ]:
from transformers import TFBertModel

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = bert_model.bert(input_ids, attention_mask=attention_masks)[1]
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(20, activation='softmax', name='output_layer')(intermediate_layer)

model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 256, 7                                     

In [ ]:
# vid doesn't use legacy Adam
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
100/100 [==============================] - 37s 230ms/step - loss: 3.0110 - accuracy: 0.0975 - val_loss: 2.7463 - val_accuracy: 0.1488
Epoch 2/10
100/100 [==============================] - 26s 265ms/step - loss: 2.7500 - accuracy: 0.1388 - val_loss: 2.6519 - val_accuracy: 0.2125
Epoch 3/10
100/100 [==============================] - 20s 198ms/step - loss: 2.6580 - accuracy: 0.1713 - val_loss: 2.4820 - val_accuracy: 0.2975
Epoch 4/10
100/100 [==============================] - 26s 260ms/step - loss: 2.5343 - accuracy: 0.2338 - val_loss: 2.2939 - val_accuracy: 0.3100
Epoch 5/10
100/100 [==============================] - 22s 215ms/step - loss: 2.3170 - accuracy: 0.2988 - val_loss: 1.9908 - val_accuracy: 0.4625
Epoch 6/10
100/100 [==============================] - 21s 209ms/step - loss: 2.0618 - accuracy: 0.3950 - val_loss: 1.7061 - val_accuracy: 0.5325
Epoch 7/10
100/100 [==============================] - 21s 212ms/step - loss: 1.7969 - accuracy: 0.4963 - val_loss: 1.4376 - val_ac

In [ ]:
!mkdir -p saved_model
model.save('saved_model/magic20model')